# Notebook 4: Inference and App Preparation

### Objectives:
1.  **Load the Best Model:** Load the saved weights of our winning model (`ResNetUNet`).
2.  **Create an Inference Pipeline:** Develop a robust function that takes a single image file, preprocesses it, and runs it through the model to get a segmentation mask.
3.  **Test the Pipeline:** Verify that our inference function works correctly on a sample image, creating the final logic we'll use in our Streamlit application.

## 1. Setup, Imports, and Path Definitions

First, we import the necessary libraries and define our project paths. We also import our model architectures from the `src` directory.

In [5]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt

# --- Define Project Directories ---
try:
    ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
except NameError:
    ROOT_DIR = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))

# --- Add Project Root to Python Path ---
# This is the crucial step that allows us to import from the 'src' directory
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

print(f"Project Root added to path: {ROOT_DIR}")

# --- Import our custom models (this will now work)---
from src.models import BaselineUNet, ResNetUNet, TransUNet

MODELS_DIR = os.path.join(ROOT_DIR, 'models')
FIGURES_DIR = os.path.join(ROOT_DIR, 'figures')

# --- Set Device ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Root Directory: {ROOT_DIR}")
print(f"Models Directory: {MODELS_DIR}")
print(f"Device: {DEVICE}")

Project Root added to path: d:\Coding\GitHub\MRI-Tumor-Segmentation
Root Directory: d:\Coding\GitHub\MRI-Tumor-Segmentation
Models Directory: d:\Coding\GitHub\MRI-Tumor-Segmentation\models
Device: cuda


## 2. Load the Best Trained Model

Based on our benchmark results, the `ResNetUNet` was the clear winner. We will now load its saved weights into a model instance and set it to evaluation mode.

In [6]:
BEST_MODEL_NAME = 'ResNetUNet'
model_path = os.path.join(MODELS_DIR, f'{BEST_MODEL_NAME}_best_model.pth')

# Instantiate the model architecture
model = ResNetUNet(in_channels=4, out_channels=1).to(DEVICE)

# Load the saved weights
model.load_state_dict(torch.load(model_path, map_location=torch.device(DEVICE)))

# Set the model to evaluation mode
model.eval()

print(f"Successfully loaded best model: {BEST_MODEL_NAME} from {model_path}")

Successfully loaded best model: ResNetUNet from d:\Coding\GitHub\MRI-Tumor-Segmentation\models\ResNetUNet_best_model.pth


## 3. Create the Inference Pipeline

Here we create a single, powerful function called `predict`. This function encapsulates all the necessary steps: opening an image, preprocessing it to match the training format, running inference, and post-processing the output mask. This is the core function we will use in our Streamlit app.

In [7]:
# =========================================================================
#             CREATE THE INFERENCE PIPELINE (CORRECTED)
# =========================================================================

def predict(model, npy_image_path, device):
    """
    Runs the full inference pipeline on a single, preprocessed .npy image slice.
    """
    # Load the correctly preprocessed numpy array
    img_np = np.load(npy_image_path)
    
    # Convert to PyTorch tensor and add batch dimension
    input_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        # Get raw prediction (logits)
        logits = model(input_tensor)
        # Convert to probabilities -> binary mask
        pred_mask = (torch.sigmoid(logits) > 0.5).float()
        
    # Remove batch dimension and move to CPU
    pred_mask = pred_mask.squeeze(0).cpu().numpy()
    
    return pred_mask

## 4. Creating Sample Images for App Deployment & Testing the Pipeline
This cell takes a few images from our test set and saves them as user-friendly .png files for the application demo.

In [8]:
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
import json

# Define the directory to save sample images
SAMPLE_IMAGES_DIR = "../streamlit_app/sample_images"
os.makedirs(SAMPLE_IMAGES_DIR, exist_ok=True)

# --- This is the updated logic ---

# Get a list of all possible .npy files from your processed data
processed_files = glob.glob("../data/processed/*_image.npy")
sample_mapping = {}
num_samples_to_create = 5 # You can change this number

print(f"--- Creating {num_samples_to_create} sample files... ---")

for i in range(num_samples_to_create):
    # 1. Select a random .npy file
    random_npy_path = random.choice(processed_files)
    img_array = np.load(random_npy_path)

    # 2. Define a new, simple filename for the .npy file
    new_npy_filename = f"sample_{i+1}_image.npy"
    new_npy_path = os.path.join(SAMPLE_IMAGES_DIR, new_npy_filename)

    # 3. Save the actual .npy array into the streamlit_app/sample_images directory
    np.save(new_npy_path, img_array)
    print(f"Saved model input file: {new_npy_path}")

    # 4. Update the mapping to use the new simple filename
    sample_mapping[f"Sample {i+1}"] = new_npy_filename


# --- Save the updated mapping file ---

mapping_path = os.path.join(SAMPLE_IMAGES_DIR, 'sample_mapping.json')
with open(mapping_path, 'w') as f:
    json.dump(sample_mapping, f, indent=2)

print(f"\n--- Sample mapping saved to: {mapping_path} ---")
print("\n--- All sample files created successfully. ---")

--- Creating 5 sample files... ---
Saved model input file: ../streamlit_app/sample_images\sample_1_image.npy
Saved model input file: ../streamlit_app/sample_images\sample_2_image.npy
Saved model input file: ../streamlit_app/sample_images\sample_3_image.npy
Saved model input file: ../streamlit_app/sample_images\sample_4_image.npy
Saved model input file: ../streamlit_app/sample_images\sample_5_image.npy

--- Sample mapping saved to: ../streamlit_app/sample_images\sample_mapping.json ---

--- All sample files created successfully. ---


## End of Notebook 4

This concludes our inference preparation. We have successfully:
- Loaded our best-performing `ResNetUNet` model.
- Built and tested a robust `predict` function that handles the full preprocessing and inference pipeline.

We are now fully prepared to build the interactive Streamlit application.